In [1]:
%load_ext autoreload
%autoreload 2

import os
import json
import copy
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
from sklearn.linear_model import LinearRegression
import ray_results_interpreter as rri
import subprocess
import concurrent.futures
from main_run import MainRun

/user/ml4723/.conda/envs/neural_inventory_control/lib/python3.10/site-packages/pandas/core/computation/expressions.py:21: UserWarning: Pandas requires version '2.8.4' or newer of 'numexpr' (version '2.8.3' currently installed).
  from pandas.core.computation.check import NUMEXPR_INSTALLED
/user/ml4723/.conda/envs/neural_inventory_control/lib/python3.10/site-packages/pandas/core/arrays/masked.py:60: UserWarning: Pandas requires version '1.3.6' or newer of 'bottleneck' (version '1.3.5' currently installed).
  from pandas.core import (


In [2]:
# Create vanilla results dataframe from the provided data
testset_name = "finals_one_store_lost"

results_interpretor = rri.RayResultsinterpreter()

def custom_data_filler(out_row, reference_row):
    out_row['path'] = reference_row['path']

def default_condition_setter(condition_name):
    return None

architectures = {
    "Vanilla NN": lambda config: f'/user/ml4723/Prj/NIC/ray_results/{testset_name}/vanilla_one_store_2',
}

sort_by = 'dev_loss'
pick_row_from_run_by = 'dev_loss'

config = "one_store_backlogged"
store_lead_times = [1, 2, 3, 4]
store_underage_costs = [4, 9, 19, 39]

dfs = []

for arch_name, path_fn in architectures.items():
    path = path_fn(config)

    df = results_interpretor.make_table({1: path},
        {'store_underage_cost': store_underage_costs,
            'store_lead_time': store_lead_times,
            'train_batch_size': [1024, 8192],
            'learning_rate': [0.01, 0.001, 0.0001]},
        default_condition_setter, custom_data_filler,
        sort_by=sort_by, pick_row_from_run_by=pick_row_from_run_by, test_loss_limit=25)
    if df.empty:
        continue

    df.insert(2, 'Architecture Class', arch_name)
    df.insert(1, 'hyperparam_name', arch_name)
    df['config'] = config
    dfs.append(df)

if not dfs:
    raise ValueError("No dataframes found for the given settings.")

df = pd.concat(dfs, ignore_index=True)

In [4]:
import numpy as np

test_loss_column = 'Test Loss'
test_loss_filename = 'one_store_lost_test_loss.txt'

test_losses = []
for _, row in df.iterrows():
    test_loss_path = str(row['path']) + "/" + test_loss_filename
    try:
        with open(test_loss_path, 'r') as f:
            value = float(f.read().strip())
    except Exception:
        value = np.nan
    test_losses.append(value)

df[test_loss_column] = test_losses

In [5]:
# Prepare a table of optimal losses for each (lead_time, underage_cost)
lead_times = [1, 2, 3, 4]
underage_costs = [4, 9, 19, 39]

# Optimal losses from the provided table, indexed as [underage_cost][lead_time]
optimal_losses_table = [
    [4.04, 4.40, 4.60, 4.73],   # p=4
    [5.44, 6.09, 6.53, 6.84],   # p=9
    [6.68, 7.66, 8.36, 8.89],   # p=19
    [7.84, 9.11, 10.04, 10.79], # p=39
]

import pandas as pd

optimal_losses_df = pd.DataFrame(
    optimal_losses_table,
    index=underage_costs,
    columns=lead_times
)
optimal_losses_df.index.name = "Underage Cost"
optimal_losses_df.columns.name = "Lead Time"

display(optimal_losses_df)


Lead Time,1,2,3,4
Underage Cost,,,,
4,4.04,4.40,4.60,4.73
9,5.44,6.09,6.53,6.84
19,6.68,7.66,8.36,8.89
39,7.84,9.11,10.04,10.79


In [6]:
import numpy as np
import pandas as pd
import os

lead_time_col = "store_lead_time"
underage_cost_col = "store_underage_cost"
dev_loss_col = "Dev Loss"
hyperparam_col = "hyperparam_name"
hidden_layers_col = "hidden_layers"
batch_size_col = "train_batch_size"
learning_rate_col = "learning_rate"
base_stock_hyperparam = "base_stock"

# Only consider vanilla NN runs (not base_stock)
vanilla_df = df[df[hyperparam_col] != base_stock_hyperparam].copy()

# Add hidden_layers column based on hyperparam_name or config
def get_hidden_layers(row):
    if "vanilla_one_store_2" in architectures['Vanilla NN'](config).split('/')[-1]:
        return 2
    if "vanilla_one_store" in architectures['Vanilla NN'](config).split('/')[-1]:
        return 3
    return np.nan

if "hidden_layers" not in vanilla_df.columns:
    vanilla_df["hidden_layers"] = vanilla_df.apply(get_hidden_layers, axis=1)

# Get all unique hyperparameter settings
group_cols = ["hidden_layers", batch_size_col, learning_rate_col]
summary_rows = []

# For each hyperparameter setting, collect all 16 (lead_time, underage_cost) instances
test_loss_col = "Test Loss"
for (hidden_layers, batch_size, learning_rate), group in vanilla_df.groupby(group_cols):
    solved_count = 0
    grad_steps_list = []
    time_to_1pct_list = []
    for _, row in group.iterrows():
        lead_time = int(row[lead_time_col])
        underage_cost = int(row[underage_cost_col])
        # Get optimal loss for this instance
        try:
            optimal_loss = optimal_losses_df.loc[underage_cost, lead_time]
        except Exception:
            continue

        # Use test loss for solved count
        test_loss = row.get(test_loss_col, np.nan)
        if not np.isnan(test_loss):
            test_gap = 100 * (test_loss - optimal_loss) / optimal_loss
            if test_gap < 0.25:
                solved_count += 1

        progress_path = os.path.join(row["path"], "progress.csv")
        try:
            progress_df = pd.read_csv(progress_path)
            dev_gap_threshold = 0.01 * optimal_loss
            first_time = progress_df["time_total_s"].iloc[0]
            first_step = progress_df["training_iteration"].iloc[0]
            found = False
            for idx, prog_row in progress_df.iterrows():
                prog_dev_gap = abs(prog_row["dev_loss"] - optimal_loss)
                if prog_dev_gap <= dev_gap_threshold:
                    grad_steps = prog_row["training_iteration"] - first_step
                    time_to_1pct = prog_row["time_total_s"] - first_time
                    # Multiply grad_steps by 10, and by 4 if batch_size is 8192, by 32 if 1024
                    batch_size_val = row[batch_size_col]
                    grad_steps_scaled = grad_steps * 10
                    if batch_size_val == 8192:
                        grad_steps_scaled *= 4
                    elif batch_size_val == 1024:
                        grad_steps_scaled *= 32
                    grad_steps_list.append(grad_steps_scaled)
                    time_to_1pct_list.append(time_to_1pct)
                    found = True
                    break
            if not found:
                grad_steps_list.append(np.nan)
                time_to_1pct_list.append(np.nan)
        except Exception:
            grad_steps_list.append(np.nan)
            time_to_1pct_list.append(np.nan)

    avg_grad_steps = int(np.nanmean(grad_steps_list)) if len(grad_steps_list) > 0 and not np.all(np.isnan(grad_steps_list)) else ""
    avg_time_to_1pct = int(np.nanmean(time_to_1pct_list)) if len(time_to_1pct_list) > 0 and not np.all(np.isnan(time_to_1pct_list)) else ""
    summary_rows.append([
        int(hidden_layers),
        int(batch_size),
        float(learning_rate),
        solved_count,
        avg_grad_steps,
        avg_time_to_1pct
    ])

summary_df = pd.DataFrame(summary_rows, columns=[
    "Hidden layers",
    "Batch size",
    "Learning rate",
    "Instances solved to optimality (#)",
    "Average gradient steps to 1% dev gap",
    "Average time to 1% dev gap (s)"
])

summary_df = summary_df.sort_values(["Hidden layers", "Batch size", "Learning rate"]).reset_index(drop=True)
summary_df

,Hidden layers,Batch size,Learning rate,Instances solved to optimality (#),Average gradient steps to 1% dev gap,Average time to 1% dev gap (s)
0,2,1024,0.0001,16,6120,467
1,2,1024,0.0010,16,1740,133
2,2,1024,0.0100,16,1420,105
3,2,8192,0.0001,15,5795,819
4,2,8192,0.0010,16,1412,203
5,2,8192,0.0100,16,1250,171


In [23]:
import numpy as np
import pandas as pd
import os

lead_time_col = "store_lead_time"
underage_cost_col = "store_underage_cost"
dev_loss_col = "Dev Loss"
hyperparam_col = "hyperparam_name"
hidden_layers_col = "hidden_layers"
batch_size_col = "train_batch_size"
learning_rate_col = "learning_rate"
base_stock_hyperparam = "base_stock"

target_hidden_layers = 3
target_batch_size = 1024
target_learning_rate = 0.01

test_loss_col = "Test Loss"

# Only consider vanilla NN runs (not base_stock)
vanilla_df = df[df[hyperparam_col] != base_stock_hyperparam].copy()

# Add hidden_layers column based on hyperparam_name or config
def get_hidden_layers(row):
    if "vanilla_one_store_2" in architectures['Vanilla NN'](config).split('/')[-1]:
        return 2
    if "vanilla_one_store" in architectures['Vanilla NN'](config).split('/')[-1]:
        return 3
    return np.nan

if "hidden_layers" not in vanilla_df.columns:
    vanilla_df["hidden_layers"] = vanilla_df.apply(get_hidden_layers, axis=1)

# Filter for the best hyperparameter setting: 3 layers, 1024 batch size, 0.01 learning rate
best_df = vanilla_df[
    (vanilla_df["hidden_layers"] == target_hidden_layers) &
    (vanilla_df[batch_size_col] == target_batch_size) &
    (np.isclose(vanilla_df[learning_rate_col], target_learning_rate))
].copy()

rows = []
for _, row in best_df.iterrows():
    lead_time = int(row[lead_time_col])
    underage_cost = int(row[underage_cost_col])
    try:
        optimal_loss = optimal_losses_df.loc[underage_cost, lead_time]
    except Exception:
        continue

    test_loss = row.get(test_loss_col, np.nan)
    if np.isnan(test_loss):
        continue
    test_gap = 100 * (test_loss - optimal_loss) / optimal_loss

    # Test gap string formatting
    test_gap_str = "<0.25" if test_gap < 0.25 else f"{test_gap:.2f}"

    progress_path = os.path.join(row["path"], "progress.csv")
    grad_steps_to_1pct = ""
    time_to_1pct = ""
    try:
        progress_df = pd.read_csv(progress_path)
        dev_gap_threshold = 0.01 * optimal_loss
        first_time = progress_df["time_total_s"].iloc[0]
        first_step = progress_df["training_iteration"].iloc[0]
        found = False
        for idx, prog_row in progress_df.iterrows():
            prog_dev_gap = abs(prog_row["dev_loss"] - optimal_loss)
            if prog_dev_gap <= dev_gap_threshold:
                grad_steps = prog_row["training_iteration"] - first_step
                time_to_1pct = prog_row["time_total_s"] - first_time
                grad_steps_scaled = grad_steps * 10 * 32  # 10x for epoch, 32x for batch size 1024
                grad_steps_to_1pct = int(grad_steps_scaled)
                time_to_1pct = int(time_to_1pct)
                found = True
                break
        if not found:
            grad_steps_to_1pct = ""
            time_to_1pct = ""
    except Exception:
        grad_steps_to_1pct = ""
        time_to_1pct = ""

    rows.append([
        lead_time,
        underage_cost,
        round(test_loss, 2),
        test_gap_str,
        grad_steps_to_1pct,
        time_to_1pct
    ])

table7_df = pd.DataFrame(rows, columns=[
    "Store lead time",
    "Store underage cost",
    "Test loss",
    "Test gap (%)",
    "Gradient steps to 1% dev gap",
    "Time to 1% dev gap (s)"
])

table7_df = table7_df.sort_values(["Store lead time", "Store underage cost"]).reset_index(drop=True)
table7_df

,Store lead time,Store underage cost,Test loss,Test gap (%),Gradient steps to 1% dev gap,Time to 1% dev gap (s)
0,1,4,4.04,<0.25,640,44
1,1,9,5.44,<0.25,640,45
2,1,19,6.68,<0.25,320,24
3,1,39,7.84,<0.25,320,23
4,2,4,4.40,<0.25,320,27
5,2,9,6.09,<0.25,1280,111
6,2,19,7.67,<0.25,960,80
7,2,39,9.11,<0.25,640,56
8,3,4,4.60,<0.25,960,82
9,3,9,6.53,<0.25,960,88


In [3]:
mode = "test"
setting_names = ['one_store_lost']

models = []
for _, row in df.iterrows():
    models.append(str(row['path']) + '/model.pt')

gpus = [0, 1, 2, 3]

import nest_asyncio
import asyncio

nest_asyncio.apply()

async def run_main_run(model_path, setting_name, gpu_idx, semaphore):
    async with semaphore:
        try:
            hyperparam_name = model_path.split('/')[7]
            print(f"Running main_run.py for path {model_path}")
            cmd = [
                "/user/ml4723/.conda/envs/neural_inventory_control/bin/python",
                "main_run.py",
                mode,
                setting_name,
                hyperparam_name,
                str(model_path),
                str(gpus[gpu_idx])
            ]
            env = {
                **os.environ,
                "MKL_THREADING_LAYER": "GNU",
                "MKL_SERVICE_FORCE_INTEL": "1"
            }
            process = await asyncio.create_subprocess_exec(
                *cmd,
                env=env,
                cwd="/user/ml4723/Prj/NIC/",
                stdout=asyncio.subprocess.PIPE,
                stderr=asyncio.subprocess.PIPE
            )
            stdout, stderr = await process.communicate()
            if process.returncode != 0:
                print(f"Error running main_run.py for path {model_path}: {stderr.decode()}")
        except Exception as e:
            print(f"Unexpected error running main_run.py for path {model_path}: {e}")

async def main():
    max_concurrent = 6 * len(gpus)
    semaphore = asyncio.Semaphore(max_concurrent)
    tasks = []
    gpu_idx = 0
    for setting_name in setting_names:
        for path in models:
            tasks.append(run_main_run(path, setting_name, gpu_idx, semaphore))
            gpu_idx = (gpu_idx + 1) % len(gpus)
    await asyncio.gather(*tasks)

await main()


Running main_run.py for path /user/ml4723/Prj/NIC/ray_results/finals_one_store_lost/vanilla_one_store_2/run_2025-05-23_13-59-42/run_b3bea_00002_2_config=one_store_lost,early_stop_check_epochs=10,learning_rate=0.0001,repeats=1,stop_if_no_improve_for_epochs=50_2025-05-23_13-59-43/model.pt
Running main_run.py for path /user/ml4723/Prj/NIC/ray_results/finals_one_store_lost/vanilla_one_store_2/run_2025-05-23_13-59-42/run_b3bea_00001_1_config=one_store_lost,early_stop_check_epochs=10,learning_rate=0.0010,repeats=1,stop_if_no_improve_for_epochs=50_2025-05-23_13-59-43/model.pt
Running main_run.py for path /user/ml4723/Prj/NIC/ray_results/finals_one_store_lost/vanilla_one_store_2/run_2025-05-23_13-59-42/run_b3bea_00000_0_config=one_store_lost,early_stop_check_epochs=10,learning_rate=0.0100,repeats=1,stop_if_no_improve_for_epochs=50_2025-05-23_13-59-43/model.pt
Running main_run.py for path /user/ml4723/Prj/NIC/ray_results/finals_one_store_lost/vanilla_one_store_2/run_2025-05-23_13-59-42/run_b3be